In [1]:
# Enable outputting results interactively. 
# All the results from code in a given cell will be displayed instead of only the last one.

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [10]:
import pandas as pd
import pickle
import numpy as np
import sqlite3
import math
import datetime

from re import sub
from decimal import Decimal

import re

import matplotlib.pyplot as plt
%matplotlib inline

import os
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

#to make the interactive maps
import folium
from folium.plugins import FastMarkerCluster
import geopandas as gpd
from branca.colormap import LinearColormap

import plotly.express as px

In [3]:
calendar =  pickle.load( open( "../data/all_SF_calendar.p", "rb" ) )
calendar.shape
# 

# limit to listings where the min. nights is less than 30 days. listings with 30 days + requirement are exempt from the regulation.


(50724783, 9)

In [4]:
calendar['price'] = calendar['price'].str.replace("\$|,", "").astype(float)
calendar.dtypes

listing_id                  int64
date               datetime64[ns]
available                  object
price                     float64
adjusted_price             object
minimum_nights            float64
maximum_nights            float64
file_scraped_dt    datetime64[ns]
lead_time                   int64
dtype: object

In [7]:
calendar_june = calendar[calendar['file_scraped_dt'] == datetime.date(2018,8,6)]
calendar_june.shape

(1073584, 9)

In [28]:
sum_available = calendar_june.groupby('date').size().to_frame(name= 'Available Number of Listings').reset_index()
sum_available['weekday'] = sum_available['date'].dt.day_name()
# sum_available = sum_available.set_index('date')



In [29]:
sum_available.head()

,date,Available Number of Listings,weekday
0,2018-08-06,581,Monday
1,2018-08-07,760,Tuesday
2,2018-08-08,822,Wednesday
3,2018-08-09,822,Thursday
4,2018-08-10,732,Friday


In [30]:
# sum_available.iplot(y='available', mode = 'lines', xTitle = 'Date', yTitle = 'number of listings available',\
#                    text='weekday', title = 'Number of listings available by date')

fig = px.line(sum_available, x="date", y="Available Number of Listings", title='Number of listings available by date in June 2019')

fig.show()
